In [2]:
# imports
from dotenv import load_dotenv
import openai
import os

from langchain.document_loaders import WebBaseLoader, UnstructuredFileLoader, TextLoader, DirectoryLoader, PyPDFLoader, UnstructuredXMLLoader

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_google_vertexai import VertexAI

from langchain import PromptTemplate, LLMChain
from langchain.document_loaders.merge import MergedDataLoader

from IPython.display import Markdown as md

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv(".env")

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

from langchain_google_vertexai.model_garden import ChatAnthropicVertex

model_name="gemini-2.5-pro-preview-05-06"
llm = VertexAI(
    model_name=model_name,
    temperature=0,
    top_p=1.0,)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
def load_and_construct(directory_paths: list, file_extensions: list) -> str:
    """
    Lädt Dateien aus einer Liste von Verzeichnissen mit bestimmten Dateiendungen,
    extrahiert Metadaten und Inhalt und gibt einen zusammengefassten String zurück.

    Args:
        directory_paths (list): Eine Liste von Pfaden zu den Verzeichnissen.
        file_extensions (list): Eine Liste der zu ladenden Dateiendungen (ohne Punkt).

    Returns:
        str: Ein String, der die Metadaten und Inhalte der Dateien zusammenfasst.
    """

    all_loaders = []
    for directory_path in directory_paths:
        for ext in file_extensions:
            all_loaders.append(
                DirectoryLoader(
                    path=directory_path,
                    glob=f"**/*.{ext}",  # Verwende "**/" für rekursive Suche in allen Unterverzeichnissen
                    recursive=True,
                    loader_cls=TextLoader,
                    show_progress=True # Optional: Zeigt einen Fortschrittsbalken an
                )
            )

    merged_loader = MergedDataLoader(loaders=all_loaders)
    raw_documents = merged_loader.load()

    result_string = ""
    for document in raw_documents:
        # Extrahiere den Dateinamen aus dem vollständigen Pfad in den Metadaten
        filepath = document.metadata["source"]
        result_string += f"Filepath: {filepath}\n"
        result_string += f"{document.page_content}\n"
        result_string += "=====================================\n"

    return result_string

In [9]:
# src_dir = load_and_construct(["/home/randy/Projekte/gdg/vertex-ai/"], ["py", "md"])

cloud_run_tf_doc = WebBaseLoader("https://github.com/hashicorp/terraform-provider-google/blob/main/website/docs/r/cloud_run_service.html.markdown").load()[0].page_content

prompt_template = """
Du bist ein DevOps-Engineer, und sollst einen vollständigen Terraform-Code schreiben um das Image docker.io/library/nginx:latest in einer Cloud Run Instanz zu deployen.

Der Code soll so geschrieben werden, dass der Benutzer nur 'terraform apply' ausführen muss und dann wir der Cloud Run Service erstellt. Der Zugriff auf Cloud Run soll ohne Authentifizierung möglich sein. Nach dem Deployment soll terraform output den URL des Cloud Run Services ausgeben.

Das Zielprojekt soll per Kommandozeilen parameter dem Terraform apply hinzugefügt werden.

Erstelle eine vollumfängliche Beschreibung, alle notwendigen Dateien mit vollqualifizierten Pfaden und einer vollständigen README.md Datei enthält. In der README soll erklärt werden, wie der Dienst per terraform erstellt, aufgerufen und per Terraform wieder gelöscht wird.

Hier findest Du die vollständige Dokumentation für den Cloud Run Service in Terraform:

{cloud_run_tf_doc}

Halte Dich bei Deiner Lösung streng an die Dokumentation des Terraform Providers.

Deine Antwort auf englisch im Markdown-Format:
"""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True
)

output = llm_chain.predict(cloud_run_tf_doc=cloud_run_tf_doc)

from datetime import datetime

# Erhalte den aktuellen Zeitstempel
timestamp = datetime.now().strftime('%Y-%m-%d-%H%M')


# Erstelle den Dateinamen mit dem Zeitstempel
filename = f'output/{timestamp}_terraform_demo_{model_name}.md'


# Öffne die Datei und schreibe den Inhalt hinein
with open(filename, 'w') as file:
    file.write(output)
    



> Entering new LLMChain chain...
Prompt after formatting:

Du bist ein DevOps-Engineer, und sollst einen vollständigen Terraform-Code schreiben um das Image docker.io/library/nginx:latest in einer Cloud Run Instanz zu deployen.

Der Code soll so geschrieben werden, dass der Benutzer nur 'terraform apply' ausführen muss und dann wir der Cloud Run Service erstellt. Der Zugriff auf Cloud Run soll ohne Authentifizierung möglich sein. Nach dem Deployment soll terraform output den URL des Cloud Run Services ausgeben.

Das Zielprojekt soll per Kommandozeilen parameter dem Terraform apply hinzugefügt werden.

Erstelle eine vollumfängliche Beschreibung, alle notwendigen Dateien mit vollqualifizierten Pfaden und einer vollständigen README.md Datei enthält. In der README soll erklärt werden, wie der Dienst per terraform erstellt, aufgerufen und per Terraform wieder gelöscht wird.

Hier findest Du die vollständige Dokumentation für den Cloud Run Service in Terraform:






























In [11]:
# src_dir = load_and_construct(["/home/randy/Projekte/gdg/vertex-ai/"], ["py", "md"])

cloud_run_tf_doc = WebBaseLoader("https://github.com/hashicorp/terraform-provider-google/blob/main/website/docs/r/cloud_run_service.html.markdown").load()[0].page_content
terraform_project = load_and_construct(["/home/randy/Schreibtisch/tmp/tf_demo"], ["tf", "md"])
error_msg = TextLoader("/home/randy/Projekte/gdg/vertex-ai/langchain/terraform_code/error.txt").load()[0].page_content

prompt_template = """
Ich habe folgendes Terraform-Projekt: 

{terraform_project}

Wenn ich das Projekt ausführe, dann erhalte ich folgende Fehlermeldung:

{error_msg}

Analysiere das Projekt und erkläre, wie der Fehler behoben werden kann. Wenn Dateien geändert werden müssen, dann nenne die Dateien mit vollqualifizierten Pfaden und vollständigem, ungekürztem Inhalt.

Hier die Dokumentation für den Cloud Run Service in Terraform:

{cloud_run_tf_doc}

Halte Dich bei Deiner Lösung streng an die Dokumentation des Terraform Providers.

Deine Antwort auf englisch im Markdown-Format:
"""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True
)

output = llm_chain.predict(
    cloud_run_tf_doc=cloud_run_tf_doc,
    terraform_project=terraform_project,
    error_msg=error_msg
)

from datetime import datetime

# Erhalte den aktuellen Zeitstempel
timestamp = datetime.now().strftime('%Y-%m-%d-%H%M')


# Erstelle den Dateinamen mit dem Zeitstempel
filename = f'output/{timestamp}_terraform_debug_{model_name}.md'


# Öffne die Datei und schreibe den Inhalt hinein
with open(filename, 'w') as file:
    file.write(output)
    

100%|██████████| 4/4 [00:00<00:00, 2438.19it/s]
0it [00:00, ?it/s]




> Entering new LLMChain chain...
Prompt after formatting:

Ich habe folgendes Terraform-Projekt: 

Filepath: /home/randy/Schreibtisch/tmp/tf_demo/variables.tf
variable "project_id" {
  type        = string
  description = "The Google Cloud project ID where the Cloud Run service will be deployed."
}

variable "region" {
  type        = string
  description = "The Google Cloud region for the Cloud Run service."
  default     = "us-central1"
}

variable "service_name" {
  type        = string
  description = "The name for the Cloud Run service."
  default     = "nginx-public-service"
}

variable "container_image" {
  type        = string
  description = "The Docker image to deploy."
  default     = "docker.io/library/nginx:latest"
}
Filepath: /home/randy/Schreibtisch/tmp/tf_demo/versions.tf
terraform {
  required_version = ">= 1.0"

  required_providers {
    google = {
      source  = "hashicorp/google"
      version = ">= 4.40.0" # Specify a recent stable version
    }
  }
}
Filepath: